### Play around with fitting polynomials to photometry 

Import stuff

In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

from rail.raruma import plotting_functions as raruma_plot
from rail.raruma import utility_functions as raruma_util


Change this to be the root of the current PZ working area

In [ ]:
#pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'
# if that fails you can use this
# pz_dir = '/global/u2/e/echarles/dx'
pz_dir = '/Users/echarles/pz'

Get the data, and split it

In [ ]:
d = tables_io.read(f"{pz_dir}/sandbox_data/roman_rubin_9925.hdf5")
train = tables_io.sliceObj(d, slice(0, -1, 5))
test = tables_io.sliceObj(d, slice(1, -1, 5))
d.keys()

In [ ]:
np.nanmax(d['LSST_obs_g'])

Get the colors & mags

In [ ]:
train_targets, train_features = raruma_util.prepare_data_total_mag_and_colors(train, 'LSST_obs_{band}', 'ugrizy')
test_targets, test_features = raruma_util.prepare_data_total_mag_and_colors(test, 'LSST_obs_{band}', 'ugrizy')

train_mags = raruma_util.get_band_values(train, 'LSST_obs_{band}', 'ugrizy')
train_mag_errs = raruma_util.get_band_values(train, 'LSST_obs_{band}_err', 'ugrizy')


Pick some medians for the filters, note: these are just estimates

In [ ]:
filter_medians = np.array([350., 470., 620., 750., 870., 950.])
x_piv = 0.5*(filter_medians[0] + filter_medians[-1])

Compute how much each color exceeds expectations

In [ ]:
color_excess = raruma_util.color_excess(train_mags)

In [ ]:
# _poly_fits = raruma_util.polynomial_fits(filter_medians, train_mags, train_mag_errs, x_piv, 1)
# plt.scatter(_poly_fits[:,0], _poly_fits[:,1], color=cm.rainbow(train_targets), marker='.', s=1)

Do linear fits and return residuals

In [ ]:
_lin_resids = raruma_util.linear_fit_residuals(filter_medians, train_mags, x_piv)

Plot residuals as a function of redshift

In [ ]:
_ = plt.hist2d(train_targets, np.nan_to_num(_lin_resids[:,1], 0.), bins=(100, 100), norm='log')

In [ ]:
_ = plt.hist2d(train_targets, np.nan_to_num(_lin_resids[:,4], 0.), bins=(100, 100), norm='log')

Resid v resid plot

In [ ]:
_ = plt.scatter(_lin_resids[:,3], _lin_resids[:,5], color=cm.rainbow(train_targets/3.), marker='.', s=1)

Make a bitmask for postivie residuals

In [ ]:
lrmask = np.sum(((_lin_resids > 0) * np.array([1, 2, 4, 8, 16, 32])), axis=1)

In [ ]:
_ = plt.hist(lrmask, bins=np.arange(64))

In [ ]:
_ = plt.hist2d(train_targets, lrmask, bins=(np.linspace(0, 3., 101), np.arange(64)), norm="log", cmap="gray")

Plot lots of spectra, color coded by redshift

In [ ]:
_ = raruma_plot.plot_mag_spectra(filter_medians, train_mags[::100,:], train_mag_errs[::100,:], train_targets[::100])

Make a single plot as an example

In [ ]:
from numpy.polynomial import Polynomial
p = Polynomial.fit(filter_medians-x_piv, train_mags[0], deg=3)

In [ ]:
resid = train_mags[0] - p(filter_medians-x_piv)

In [ ]:
resid

In [ ]:
grid = np.linspace(filter_medians[0], filter_medians[-1], 61) 
plt.errorbar(filter_medians, train_mags[0], train_mag_errs[0])
plt.plot(grid, p(grid-x_piv))

In [ ]:
p.coef